<a href="https://colab.research.google.com/github/EmperoR1127/CSI5155_project/blob/master/csi5155_assignment_2_part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [55]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images/"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

data = arff.loadarff('/content/drive/My Drive/Data/seismic-bumps.arff')
seismic_bumps_data = pd.DataFrame(data[0])
#pre-process the train_set
train_labels = seismic_bumps_data[["class"]].copy()
train_set = seismic_bumps_data.drop(["class"], axis=1)
train_labels["class"] = train_labels["class"].map(lambda x: str(x)[2])
train_set_num = seismic_bumps_data.drop(["seismic","seismoacoustic","shift", "ghazard", "class"], axis=1)
train_set_cat = seismic_bumps_data.drop(["genergy","gpuls","gdenergy", "gdpuls", "nbumps", "nbumps2", "nbumps3", "nbumps4", "nbumps5", "nbumps6", "nbumps7", "nbumps89", "energy", "maxenergy", "class"], axis=1)
#build the pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
full_pipeline = ColumnTransformer([("num", num_pipeline, list(train_set_num)),("cat", OneHotEncoder(), list(train_set_cat)),])
#prepare the data
train_set_prepared = full_pipeline.fit_transform(train_set)
#prepare the target
encoder = LabelEncoder()
train_labels_prepared = encoder.fit_transform(train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [56]:
#rebalance the dataset using 3 different approaches
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from imblearn.combine import SMOTEENN
#rebalance the dataset using oversampling (random oversampling)
ros = RandomOverSampler(random_state=42)
ros_train_set_prepared, ros_train_labels_prepared = ros.fit_resample(train_set_prepared, train_labels_prepared)
print("Class distribution of oversampling" + str(sorted(Counter(ros_train_labels_prepared).items())))

#rebalance the dataset using undersampling (nearest neightbours)
renn = RepeatedEditedNearestNeighbours()
renn_train_set_prepared, renn_train_labels_prepared = renn.fit_resample(train_set_prepared, train_labels_prepared)
print("Class distribution of undersampling" + str(sorted(Counter(renn_train_labels_prepared).items())))

#rebalance the dataset using balanced sampling (SMOTEENN)
smote_enn = SMOTEENN(random_state=0)
smote_enn_train_set_prepared, smote_enn_train_labels_prepared = smote_enn.fit_resample(train_set_prepared, train_labels_prepared)
print("Class distribution of balanced sampling" + str(sorted(Counter(smote_enn_train_labels_prepared).items())))

Class distribution of oversampling[(0, 2414), (1, 2414)]
Class distribution of undersampling[(0, 1920), (1, 170)]
Class distribution of balanced sampling[(0, 1818), (1, 2194)]


In [57]:
#train the model using DECISION TREE algorithm with 10 fold cross validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score, accuracy_score
dt_clf = DecisionTreeClassifier()

#train the model on the oversampling dataset
dt_or_train_prediction = cross_val_predict(dt_clf.fit(train_set_prepared, train_labels_prepared), train_set_prepared, train_labels_prepared, cv=10)
#train the model on the oversampling dataset
dt_ros_train_prediction = cross_val_predict(dt_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
dt_renn_train_prediction = cross_val_predict(dt_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
dt_smote_enn_train_prediction = cross_val_predict(dt_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of original dataset
or_precision_score = precision_score(train_labels_prepared, dt_or_train_prediction)
or_recall_score = recall_score(train_labels_prepared, dt_or_train_prediction)
or_accuracy_score = accuracy_score(train_labels_prepared, dt_or_train_prediction)
print("precision of decision tree model on original dataset is + %f" % or_precision_score)
print("recall of decision tree model on original dataset is + %f" % or_recall_score)
print("accuracy of decision tree model on original dataset is + %f" % or_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of oversampling dataset
dt_ros_precision_score = precision_score(ros_train_labels_prepared, dt_ros_train_prediction)
dt_ros_recall_score = recall_score(ros_train_labels_prepared, dt_ros_train_prediction)
dt_ros_accuracy_score = accuracy_score(ros_train_labels_prepared, dt_ros_train_prediction)
print("precision of decision tree model on oversampling dataset is + %f" % dt_ros_precision_score)
print("recall of decision tree model on oversampling dataset is + %f" % dt_ros_recall_score)
print("accuracy of decision tree model on oversampling dataset is + %f" % dt_ros_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
dt_renn_precision_score = precision_score(renn_train_labels_prepared, dt_renn_train_prediction)
dt_renn_recall_score = recall_score(renn_train_labels_prepared, dt_renn_train_prediction)
dt_renn_accuracy_score = accuracy_score(renn_train_labels_prepared, dt_renn_train_prediction)
print("precision of decision tree model on undersampling dataset is + %f" % dt_renn_precision_score)
print("recall of decision tree model on undersampling dataset is + %f" % dt_renn_recall_score)
print("accuracy of decision tree model on undersampling dataset is + %f" % dt_renn_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
dt_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, dt_smote_enn_train_prediction)
dt_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, dt_smote_enn_train_prediction)
dt_smote_accuracy_score = accuracy_score(smote_enn_train_labels_prepared, dt_smote_enn_train_prediction)
print("precision of decision tree model on balanced sampling dataset is + %f" % dt_smote_enn_precision_score)
print("recall of decision tree model on balanced sampling dataset is + %f" % dt_smote_enn_recall_score)
print("accuracy of decision tree model on balanced sampling dataset is + %f" % dt_smote_accuracy_score)

precision of decision tree model on original dataset is + 0.077739
recall of decision tree model on original dataset is + 0.129412
accuracy of decision tree model on original dataset is + 0.841718
-----------------------------------------
precision of decision tree model on oversampling dataset is + 0.909913
recall of decision tree model on oversampling dataset is + 1.000000
accuracy of decision tree model on oversampling dataset is + 0.950497
-----------------------------------------
precision of decision tree model on undersampling dataset is + 0.264317
recall of decision tree model on undersampling dataset is + 0.352941
accuracy of decision tree model on undersampling dataset is + 0.867464
-----------------------------------------
precision of decision tree model on balanced sampling dataset is + 0.893009
recall of decision tree model on balanced sampling dataset is + 0.954877
accuracy of decision tree model on balanced sampling dataset is + 0.912762


In [58]:
#train the model using K NEAREST NEIGHBOURS algorithm with 10 fold cross validation
from sklearn import neighbors
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
n_neighbors = 5
knn_clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')

#train the model on the original dataset
knn_or_train_prediction = cross_val_predict(knn_clf.fit(train_set_prepared, train_labels_prepared), train_set_prepared, train_labels_prepared, cv=10)
#train the model on the oversampling dataset
knn_ros_train_prediction = cross_val_predict(knn_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
knn_renn_train_prediction = cross_val_predict(knn_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
knn_smote_enn_train_prediction = cross_val_predict(knn_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of original dataset
knn_or_precision_score = precision_score(train_labels_prepared, knn_or_train_prediction)
knn_or_recall_score = recall_score(train_labels_prepared, knn_or_train_prediction)
knn_or_accuracy_score = accuracy_score(train_labels_prepared, knn_or_train_prediction)
print("precision of k-nearest neighbours model on original dataset is + %f" % knn_or_precision_score)
print("recall of k-nearest neighbours model on original dataset is + %f" % knn_or_recall_score)
print("accuracy of k-nearest neighbours model on original dataset is + %f" % knn_or_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of oversampling dataset
knn_ros_precision_score = precision_score(ros_train_labels_prepared, knn_ros_train_prediction)
knn_ros_recall_score = recall_score(ros_train_labels_prepared, knn_ros_train_prediction)
knn_ros_accuracy_score = accuracy_score(ros_train_labels_prepared, knn_ros_train_prediction)
print("precision of k-nearest neighbours model on oversampling dataset is + %f" % knn_ros_precision_score)
print("recall of k-nearest neighbours model on oversampling dataset is + %f" % knn_ros_recall_score)
print("accuracy of k-nearest neighbours model on oversampling dataset is + %f" % knn_ros_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
knn_renn_precision_score = precision_score(renn_train_labels_prepared, knn_renn_train_prediction)
knn_renn_recall_score = recall_score(renn_train_labels_prepared, knn_renn_train_prediction)
knn_renn_accuracy_score = accuracy_score(renn_train_labels_prepared, knn_renn_train_prediction)
print("precision of k-nearest neighbours model on undersampling dataset is + %f" % knn_renn_precision_score)
print("recall of k-nearest neighbours model on undersampling dataset is + %f" % knn_renn_recall_score)
print("accuracy of k-nearest neighbours model on undersampling dataset is + %f" % knn_renn_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
knn_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, knn_smote_enn_train_prediction)
knn_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, knn_smote_enn_train_prediction)
knn_smote_enn_accuracy_score = accuracy_score(smote_enn_train_labels_prepared, knn_smote_enn_train_prediction)
print("precision of k-nearest neighbours model on balanced sampling dataset is + %f" % knn_smote_enn_precision_score)
print("recall of k-nearest neighbours model on balanced sampling dataset is + %f" % knn_smote_enn_recall_score)
print("accuracy of k-nearest neighbours model on balanced sampling dataset is + %f" % knn_smote_enn_accuracy_score)

precision of k-nearest neighbours model on original dataset is + 0.180952
recall of k-nearest neighbours model on original dataset is + 0.111765
accuracy of k-nearest neighbours model on original dataset is + 0.908282
-----------------------------------------
precision of k-nearest neighbours model on oversampling dataset is + 0.826996
recall of k-nearest neighbours model on oversampling dataset is + 1.000000
accuracy of k-nearest neighbours model on oversampling dataset is + 0.895402
-----------------------------------------
precision of k-nearest neighbours model on undersampling dataset is + 0.583333
recall of k-nearest neighbours model on undersampling dataset is + 0.329412
accuracy of k-nearest neighbours model on undersampling dataset is + 0.926316
-----------------------------------------
precision of k-nearest neighbours model on balanced sampling dataset is + 0.916351
recall of k-nearest neighbours model on balanced sampling dataset is + 0.993619
accuracy of k-nearest neighbou

In [59]:
#train the model using NAIVE BAYES algorithm with 10 fold cross validation
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
gnb_clf = GaussianNB()

#train the model on the original dataset
gnb_or_train_prediction = cross_val_predict(gnb_clf.fit(train_set_prepared, train_labels_prepared), train_set_prepared, train_labels_prepared, cv=10)
#train the model on the oversampling dataset
gnb_ros_train_prediction = cross_val_predict(gnb_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
gnb_renn_train_prediction = cross_val_predict(gnb_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
gnb_smote_enn_train_prediction = cross_val_predict(gnb_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of original dataset
gnb_or_precision_score = precision_score(train_labels_prepared, gnb_or_train_prediction)
gnb_or_recall_score = recall_score(train_labels_prepared, gnb_or_train_prediction)
gnb_or_accuracy_score = accuracy_score(train_labels_prepared, gnb_or_train_prediction)
print("precision of naive bayes model on original dataset is + %f" % gnb_or_precision_score)
print("recall of naive bayes model on original dataset is + %f" % gnb_or_recall_score)
print("accuracy of naive bayes model on original dataset is + %f" % gnb_or_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of oversampling dataset
gnb_ros_precision_score = precision_score(ros_train_labels_prepared, gnb_ros_train_prediction)
gnb_ros_recall_score = recall_score(ros_train_labels_prepared, gnb_ros_train_prediction)
gnb_ros_accuracy_score = accuracy_score(ros_train_labels_prepared, gnb_ros_train_prediction)
print("precision of naive bayes model on oversampling dataset is + %f" % gnb_ros_precision_score)
print("recall of naive bayes model on oversampling dataset is + %f" % gnb_ros_recall_score)
print("accuracy of naive bayes model on oversampling dataset is + %f" % gnb_ros_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
gnb_renn_precision_score = precision_score(renn_train_labels_prepared, gnb_renn_train_prediction)
gnb_renn_recall_score = recall_score(renn_train_labels_prepared, gnb_renn_train_prediction)
gnb_renn_accuracy_score = accuracy_score(renn_train_labels_prepared, gnb_renn_train_prediction)
print("precision of naive bayes model on undersampling dataset is + %f" % gnb_renn_precision_score)
print("recall of naive bayes model on undersampling dataset is + %f" % gnb_renn_recall_score)
print("accuracy of naive bayes model on undersampling dataset is + %f" % gnb_renn_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
gnb_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, gnb_smote_enn_train_prediction)
gnb_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, gnb_smote_enn_train_prediction)
gnb_smote_enn_accuracy_score = accuracy_score(smote_enn_train_labels_prepared, gnb_smote_enn_train_prediction)
print("precision of naive bayes model on balanced sampling dataset is + %f" % gnb_smote_enn_precision_score)
print("recall of naive bayes model on balanced sampling dataset is + %f" % gnb_smote_enn_recall_score)
print("accuracy of naive bayes model on balanced sampling dataset is + %f" % gnb_smote_enn_accuracy_score)

precision of naive bayes model on original dataset is + 0.088388
recall of naive bayes model on original dataset is + 0.900000
accuracy of naive bayes model on original dataset is + 0.382740
-----------------------------------------
precision of naive bayes model on oversampling dataset is + 0.593566
recall of naive bayes model on oversampling dataset is + 0.917150
accuracy of naive bayes model on oversampling dataset is + 0.644573
-----------------------------------------
precision of naive bayes model on undersampling dataset is + 0.118022
recall of naive bayes model on undersampling dataset is + 0.870588
accuracy of naive bayes model on undersampling dataset is + 0.460287
-----------------------------------------
precision of naive bayes model on balanced sampling dataset is + 0.698139
recall of naive bayes model on balanced sampling dataset is + 0.923428
accuracy of naive bayes model on balanced sampling dataset is + 0.739781


In [70]:
#train the model using RULE BASED algorithm with 10 fold cross validation
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
dc_clf = DummyClassifier(strategy='stratified')

#train the model on the original dataset
dc_or_train_prediction = cross_val_predict(dc_clf.fit(train_set_prepared, train_labels_prepared), train_set_prepared, train_labels_prepared, cv=10)
#train the model on the oversampling dataset
dc_ros_train_prediction = cross_val_predict(dc_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
dc_renn_train_prediction = cross_val_predict(dc_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
dc_smote_enn_train_prediction = cross_val_predict(dc_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of original dataset
dc_or_precision_score = precision_score(train_labels_prepared, dc_or_train_prediction)
dc_or_recall_score = recall_score(train_labels_prepared, dc_or_train_prediction)
dc_or_accuracy_score = accuracy_score(train_labels_prepared, dc_or_train_prediction)
print("precision of rule based model on original dataset is + %f" % dc_or_precision_score)
print("recall of rule based model on original dataset is + %f" % dc_or_recall_score)
print("accuracy of rule based model on original dataset is + %f" % dc_or_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of oversampling dataset
dc_ros_precision_score = precision_score(ros_train_labels_prepared, dc_ros_train_prediction)
dc_ros_recall_score = recall_score(ros_train_labels_prepared, dc_ros_train_prediction)
dc_ros_accuracy_score = accuracy_score(ros_train_labels_prepared, dc_ros_train_prediction)
print("precision of rule based model on oversampling dataset is + %f" % dc_ros_precision_score)
print("recall of rule based model on oversampling dataset is + %f" % dc_ros_recall_score)
print("accuracy of rule based model on oversampling dataset is + %f" % dc_ros_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
dc_renn_precision_score = precision_score(renn_train_labels_prepared, dc_renn_train_prediction)
dc_renn_recall_score = recall_score(renn_train_labels_prepared, dc_renn_train_prediction)
dc_renn_accuracy_score = accuracy_score(renn_train_labels_prepared, dc_renn_train_prediction)
print("precision of rule based model on undersampling dataset is + %f" % dc_renn_precision_score)
print("recall of rule based model on undersampling dataset is + %f" % dc_renn_recall_score)
print("accuracy of rule based model on undersampling dataset is + %f" % dc_renn_accuracy_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
dc_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, dc_smote_enn_train_prediction)
dc_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, dc_smote_enn_train_prediction)
dc_smote_enn_accuracy_score = accuracy_score(smote_enn_train_labels_prepared, dc_smote_enn_train_prediction)
print("precision of rule based model on balanced sampling dataset is + %f" % dc_smote_enn_precision_score)
print("recall of rule based model on balanced sampling dataset is + %f" % dc_smote_enn_recall_score)
print("accuracy of rule based model on balanced sampling dataset is + %f" % dc_smote_enn_accuracy_score)

precision of rule based model on original dataset is + 0.059211
recall of rule based model on original dataset is + 0.052941
accuracy of rule based model on original dataset is + 0.882353
-----------------------------------------
precision of rule based model on oversampling dataset is + 0.499384
recall of rule based model on oversampling dataset is + 0.504143
accuracy of rule based model on oversampling dataset is + 0.499379
-----------------------------------------
precision of rule based model on undersampling dataset is + 0.035088
recall of rule based model on undersampling dataset is + 0.035294
accuracy of rule based model on undersampling dataset is + 0.842584
-----------------------------------------
precision of rule based model on balanced sampling dataset is + 0.550339
recall of rule based model on balanced sampling dataset is + 0.555606
accuracy of rule based model on balanced sampling dataset is + 0.508724


In [85]:
from decimal import Decimal
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
skf = StratifiedKFold(n_splits=10)
dt_accuracy = []
knn_accuracy = []
gnb_accuracy = []
dc_accuracy = []
#create 10 trainsets and testsets
for train_index, test_index in skf.split(smote_enn_train_set_prepared, smote_enn_train_labels_prepared):
  #define each train set and test set
  X_train, X_test = smote_enn_train_set_prepared[train_index], smote_enn_train_set_prepared[test_index]
  y_train, y_test = smote_enn_train_labels_prepared[train_index], smote_enn_train_labels_prepared[test_index]
  #calculate accuracy of 4 models trained against each fold
  dt_accuracy.append(accuracy_score(y_test, dt_clf.fit(X_train, y_train).predict(X_test)))
  knn_accuracy.append(accuracy_score(y_test, knn_clf.fit(X_train, y_train).predict(X_test)))
  gnb_accuracy.append(accuracy_score(y_test, gnb_clf.fit(X_train, y_train).predict(X_test)))
  dc_accuracy.append(accuracy_score(y_test, dc_clf.fit(X_train, y_train).predict(X_test)))
print("The accuracies of the DECISION TREE model trained against the balanced sampling dataset is: ")
print(dt_accuracy)
print("-----------------------------------------")
print("The accuracies of the K NEAREST NEIGHBOURS model trained against the balanced sampling dataset is: ")
print(knn_accuracy)
print("-----------------------------------------")
print("The accuracies of the NAIVE BAYES model trained against the balanced sampling dataset is: ")
print(gnb_accuracy)
print("-----------------------------------------")
print("The accuracies of the RULE BASED  model trained against the balanced sampling dataset is: ")
print(dc_accuracy)

The accuracies of the DECISION TREE model trained against the balanced sampling dataset is: 
[0.8308457711442786, 0.835820895522388, 0.8980099502487562, 0.9303482587064676, 0.942643391521197, 0.9326683291770573, 0.9501246882793017, 0.9102244389027432, 0.945, 0.955]
-----------------------------------------
The accuracies of the K NEAREST NEIGHBOURS model trained against the balanced sampling dataset is: 
[0.8905472636815921, 0.9104477611940298, 0.9402985074626866, 0.9751243781094527, 0.9625935162094763, 0.942643391521197, 0.9800498753117207, 0.9476309226932669, 0.9375, 0.9825]
-----------------------------------------
The accuracies of the NAIVE BAYES model trained against the balanced sampling dataset is: 
[0.7562189054726368, 0.7189054726368159, 0.7935323383084577, 0.8756218905472637, 0.7182044887780549, 0.7356608478802993, 0.7356608478802993, 0.7057356608478803, 0.68, 0.6775]
-----------------------------------------
The accuracies of the RULE BASED  model trained against the balanc

In [86]:
#Determine whether there is a statistically significant difference in the accuracies obtained by the fouralgorithms.
#calculate p-value using paired t test 
from decimal import Decimal
from scipy import stats
print("The p-value of DT-KNN is")
print(Decimal(stats.ttest_rel(a=dt_accuracy,b=knn_accuracy)[1]).quantize(Decimal("0.0000")))
print("-----------------------------------------")
print("The p-value of DT-NB is")
print(Decimal(stats.ttest_rel(a=dt_accuracy,b=gnb_accuracy)[1]).quantize(Decimal("0.0000")))
print("-----------------------------------------")
print("The p-value of DT-RB is")
print(Decimal(stats.ttest_rel(a=dt_accuracy,b=dc_accuracy)[1]).quantize(Decimal("0.0000")))
print("-----------------------------------------")
print("The p-value of KNN-NB is")
print(Decimal(stats.ttest_rel(a=knn_accuracy,b=gnb_accuracy)[1]).quantize(Decimal("0.0000")))
print("-----------------------------------------")
print("The p-value of KNN-RB is")
print(Decimal(stats.ttest_rel(a=knn_accuracy,b=dc_accuracy)[1]).quantize(Decimal("0.0000")))
print("-----------------------------------------")
print("The p-value of NB-RB is")
print(Decimal(stats.ttest_rel(a=gnb_accuracy,b=dc_accuracy)[1]).quantize(Decimal("0.0000")))

The p-value of DT-KNN is
0.0015
-----------------------------------------
The p-value of DT-NB is
0.0001
-----------------------------------------
The p-value of DT-RB is
0.0000
-----------------------------------------
The p-value of KNN-NB is
0.0000
-----------------------------------------
The p-value of KNN-RB is
0.0000
-----------------------------------------
The p-value of NB-RB is
0.0000


In [0]:
#apply two different feature selection techniques to the balanced sampling dataset
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
#tree-based feature selection
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared)
tb_model = SelectFromModel(clf, prefit=True)
tb_smote_enn_train_set_prepared = tb_model.transform(smote_enn_train_set_prepared)
print("Dataset with " + str(smote_enn_train_set_prepared.shape[1]) + " features is reduced to " + str(tb_smote_enn_train_set_prepared.shape[1])
      + " features after applying tree based feature selection technique")

#L1-based feature selection
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared)
l_model = SelectFromModel(lsvc, prefit=True)
l_smote_enn_train_set_prepared = l_model.transform(smote_enn_train_set_prepared)
print("Dataset with " + str(smote_enn_train_set_prepared.shape[1]) + " features is reduced to " + str(l_smote_enn_train_set_prepared.shape[1])
      + " features after applying L1-based feature selection technique")

Dataset with 24 features is reduced to 10 features after applying tree based feature selection technique
Dataset with 24 features is reduced to 8 features after applying L1-based feature selection technique


In [0]:
#the accuracies of the DECISION TREE model trained with balanced sampling dataset applying 2 feature selection technique
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
score_method = make_scorer(accuracy_score)
print("The accuracies of the DECISION TREE model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(dt_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the DECISION TREE model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(dt_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the DECISION TREE model (balanced sampling + tree based feature selection) is: 
[0.84577114 0.84079602 0.9079602  0.96517413 0.93266833 0.9276808
 0.9201995  0.91022444 0.9175     0.945     ]
The accuracies of the DECISION TREE model (balanced sampling + L1 based feature selection) is: 
[0.82587065 0.8159204  0.89800995 0.96268657 0.93516209 0.91022444
 0.92518703 0.90274314 0.9325     0.9325    ]


In [0]:
#the accuracies of the K NEAREST NEIGHBOURS model with balanced sampling dataset applying 2 feature selection technique
print("The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(knn_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(knn_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + tree based feature selection) is: 
[0.88557214 0.87064677 0.92288557 0.96766169 0.89775561 0.94513716
 0.94264339 0.95261845 0.93       0.9725    ]
The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + L1 based feature selection) is: 
[0.81840796 0.83084577 0.89303483 0.97014925 0.90523691 0.90523691
 0.95261845 0.9127182  0.9225     0.94      ]


In [0]:
#the accuracies of the NAIVE BAYES model with balanced sampling dataset applying 2 feature selection technique
print("The accuracies of the NAIVE BAYES model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(gnb_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the NAIVE BAYES model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(gnb_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the NAIVE BAYES model (balanced sampling + tree based feature selection) is: 
[0.66666667 0.66915423 0.74626866 0.80099502 0.81296758 0.75311721
 0.7605985  0.78802993 0.73       0.765     ]
The accuracies of the NAIVE BAYES model (balanced sampling + L1 based feature selection) is: 
[0.60447761 0.66169154 0.75870647 0.81343284 0.77306733 0.78802993
 0.74563591 0.74314214 0.7375     0.7275    ]


In [0]:
#the accuracies of the RULE BASED  model with balanced sampling dataset applying 2 feature selection technique
print("The accuracies of the RULE BASED  model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(dc_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the RULE BASED  model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(dc_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the RULE BASED  model (balanced sampling + tree based feature selection) is: 
[0.47761194 0.5199005  0.45771144 0.48507463 0.48379052 0.52119701
 0.50623441 0.43890274 0.5        0.5375    ]
The accuracies of the RULE BASED  model (balanced sampling + L1 based feature selection) is: 
[0.46268657 0.50995025 0.52238806 0.50746269 0.48129676 0.48628429
 0.52618454 0.53117207 0.475      0.4875    ]
